# How To Implement Quantum Error Mitigation With Qiskit And Mitiq

**Learn how to implement the Clifford Data Regression**


This post appeared first on [Medium](https://pyqml.medium.com/mitigating-quantum-errors-using-clifford-data-regression-98ab663bf4c6) and my weekly [email-course](https://pyqml.substack.com/p/mitigating-quantum-errors-using-clifford?s=w).

Quantum error mitigation is paramount to tap the potential of quantum computing today. First, the qubits we have today suffer from noise in the environment ultimately destroying any meaningful computation. Second, by no means, we don't have enough physical qubits to bundle them into fault-tolerant logical qubits.

The best we can do today is to reduce the impact the noise has on the computation. That is what quantum error mitigation is about.

Recently, IBM announced its second Quantum Science Prize. They're looking for a solution to a quantum simulation problem. They want us to use Trotterization to simulate a 3-particle Heisenberg model Hamiltonian. Yet, the main challenge is to cope with the inevitable noise because they want us to solve the problem on their 7-qubit Jakarta system.

But before we can solve this problem on the real quantum computer, let's first have a look at how we can implement a quantum error mitigation method with Qiskit at all. In my previous post, I introduced the Clifford Data Regression (CDR) method developed by [P. Czarnik et al., Error mitigation with Clifford quantum-circuit data, Quantum 5, 592 (2021)](https://arxiv.org/abs/2005.10189). In this recent and promising error mitigation method, we create a machine learning model that we can use to predict and mitigate the noise by using the data from quantum circuits that we can simulate classically.

We use Qiskit, the IBM quantum development library, and Mitiq, a Python toolkit for implementing error mitigation techniques on quantum computers. 

Mitiq provides an API for the CDR method and they integrate well with Qiskit. So, it should be a piece of cake to get this working, no?

Let's first have a look at [Mitiq's introduction](https://mitiq.readthedocs.io/en/stable/guide/cdr-1-intro.html) to CDR. At first sight, they clearly describe what we need to do. It is a four-step procedure:

1. Define a quantum circuit
1. Define an executor
1.  Observable
1. (Near-Clifford) Simulator

However, when we look a little closer, we see that their example uses Google's Cirq library.

So, we need to adapt the code to the Qiskit API.


## Define A Quantum Circuit

The quantum circuit we need to define represents the problem we aim to solve, such as the Hamiltonian simulation IBM asks us for. Yet, we stick with the example Mitiq provides. This is a two-qubit circuit that only consists of Clifford gates and rotations around the Z-axis ($R_Z$). Clifford gates are easy to simulate on a classical computer--a precondition for the CDR method.

The following listing depicts the adaption of the quantum circuit to Qiskit.

In [45]:
# 1. Define a quantum circuit
from qiskit import QuantumCircuit

def get_circuit():
    qc = QuantumCircuit(2)

    # CDR works better if the circuit is not too short. So we increase its depth.
    for i in range(5): 
        qc.h(0) # Clifford
        qc.h(1) # Clifford
        qc.rz(1.75, 0)
        qc.rz(2.31, 1)
        qc.cx(0,1) # Clifford
        qc.rz(-1.17, 1)
        qc.rz(3.23, 0)
        qc.rx(pi/2, 0) # Clifford
        qc.rx(pi/2, 1) # Clifford

    # We need to measure the qubits
    #qc.measure_all()
    return qc

There's nothing really special going on here. Inside the for-loop, we apply some arbitrary quantum gates on our qubits. These are mainly rotations (`rz`, `rx`, and entanglements `cx`). For the application of the CDR method, the details of this circuit do not really matter. As mentioned, let's assume they represent the problem at hand.

Similar to Mitiq's example, we apply the series of gates multiple times to increase the length of the overall circuit. In fact, if we look ahead to solving the Hamiltonian simulation using Trotterization, this structure comes in handy because Trotterization builds upon such a repetition of a series of quantum gates.

Finally, the important difference to the example is the measurement we include in our circuit. In Qiskit, we need to specify explicitly when we "look" at our qubits.

## Define An Executor

In the next step, we need to define an executor. This is a function that takes our quantum circuit as the input and returns a Mitiq QuantumResult. Sounds easy. Yet, the devil is in the detail.

When we look at the exemplary code, we see that it uses the Mitiq function compute_density_matrix that they import from mitiq.interface.mitiq_cirq. Apparently, it returns the density matrix. This is a matrix that describes the quantum state.

Unfortunately, when we look at [Mitiq's API documentation](https://mitiq.readthedocs.io/en/stable/apidoc.html), there is no such function anymore. It seems as if the example is a little outdated. A look at the [actual source code](https://github.com/unitaryfund/mitiq/tree/master/mitiq/interface/mitiq_cirq) confirms this assumption. There is no such function anymore.

Instead, Mitiq now provides four Cirq-related functions: `execute`, `execute_with_depolarizing_noise`, `execute_with_shots`, and `execute_with_shots_and_depolarizing_noise`.

The same accounts for their Qiskit interface. Here, there are `execute`, `execute_with_noise`, `execute_with_shots`, and `execute_with_shots_and_noise`.

The question is: Which one should we use?

In the Mitiq example, they say that they added single-qubit depolarizing noise. So, we certainly want to create an executor with noise. But, do we need multiple shots?

The answer is: yes, we do! In the original example, they return the final density matrix-a representation of a quantum state. If we run the circuit only once (without shots), we won't be able to create such a matrix.

So, this is the function we're going to use:

Do you notice anything? Right, the function returns a float, not a density matrix. Furthermore, the function requires an `obs` parameter. This is an observable as a NumPy array. We would create the observable in the next step. So, let's postpone the definition of the executor for a second.

## Observable

Generally, the observable is something we can measure. But, let's not get into the physical details too much. Rather, let's look at it from a conceptual perspective.

A qubit is a two-dimensional system as depicted in the following image. The poles of the visualization depict the basis states $|0\rangle$ and $|1\rangle$. The arrow is the quantum state vector. The proximities to the poles (the basis states) denote the amplitudes whose squares are the probabilities of measuring the qubit as either 0 or 1. Simply put, the closer the quantum state vector is to the basis state $|1\rangle$ the higher the probability of measuring the qubit as a 1.

So far so good. Yet, the amplitudes of the quantum state are complex numbers. A complex number is a two-dimensional number with a real part and an imaginary part as shown in the following figure.

This effectively turns the qubit into a three-dimensional construct that we usually represent as the Bloch Sphere. Still, the proximities to the poles determine the measurement probabilities.


A sphere is homogeneous. There's no special point at all. The definition of the poles to represent $|0\rangle$ and $|1\rangle$ is arbitrary. We could define two other opposing points on the sphere's surface and ask for the probabilities of measuring the qubit as either one. The following figure depicts two such points.


\begin{figure}[h]
\begin{board}
\centering



\tikzset{every picture/.style={line width=0.75pt}} %set default line width to 0.75pt        

\begin{tikzpicture}[x=0.75pt,y=0.75pt,yscale=-1,xscale=1]
%uncomment if require: \path (0,300); %set diagram left start at 0, and has height of 300

%Shape: Ellipse [id:dp3658152106146677] 
\draw  [color={rgb, 255:red, 255; green, 255; blue, 255 }  ,draw opacity=1 ][line width=0.75]  (87,71.06) .. controls (87,51.73) and (93.27,36.06) .. (101,36.06) .. controls (108.73,36.06) and (115,51.73) .. (115,71.06) .. controls (115,90.39) and (108.73,106.06) .. (101,106.06) .. controls (93.27,106.06) and (87,90.39) .. (87,71.06) -- cycle ;
%Shape: Circle [id:dp07761280290757333] 
\draw  [color={rgb, 255:red, 255; green, 255; blue, 255 }  ,draw opacity=1 ][line width=0.75]  (66,71.06) .. controls (66,51.73) and (81.67,36.06) .. (101,36.06) .. controls (120.33,36.06) and (136,51.73) .. (136,71.06) .. controls (136,90.39) and (120.33,106.06) .. (101,106.06) .. controls (81.67,106.06) and (66,90.39) .. (66,71.06) -- cycle ;
%Shape: Ellipse [id:dp46765485835014275] 
\draw  [color={rgb, 255:red, 255; green, 255; blue, 255 }  ,draw opacity=1 ][line width=0.75]  (101,57.06) .. controls (120.33,57.06) and (136,63.32) .. (136,71.06) .. controls (136,78.79) and (120.33,85.06) .. (101,85.06) .. controls (81.67,85.06) and (66,78.79) .. (66,71.06) .. controls (66,63.32) and (81.67,57.06) .. (101,57.06) -- cycle ;
%Straight Lines [id:da24555337677859002] 
\draw [color={rgb, 255:red, 255; green, 255; blue, 255 }  ,draw opacity=1 ][line width=1.5]    (101,102.06) -- (101,40.06) ;
\draw [shift={(101,36.06)}, rotate = 90] [fill={rgb, 255:red, 255; green, 255; blue, 255 }  ,fill opacity=1 ][line width=0.08]  [draw opacity=0] (11.61,-5.58) -- (0,0) -- (11.61,5.58) -- cycle    ;
\draw [shift={(101,106.06)}, rotate = 270] [fill={rgb, 255:red, 255; green, 255; blue, 255 }  ,fill opacity=1 ][line width=0.08]  [draw opacity=0] (11.61,-5.58) -- (0,0) -- (11.61,5.58) -- cycle    ;
%Shape: Ellipse [id:dp21984344557821944] 
\draw  [color={rgb, 255:red, 255; green, 255; blue, 255 }  ,draw opacity=1 ][line width=0.75]  (188,70.06) .. controls (188,50.73) and (194.27,35.06) .. (202,35.06) .. controls (209.73,35.06) and (216,50.73) .. (216,70.06) .. controls (216,89.39) and (209.73,105.06) .. (202,105.06) .. controls (194.27,105.06) and (188,89.39) .. (188,70.06) -- cycle ;
%Shape: Circle [id:dp54831823072177] 
\draw  [color={rgb, 255:red, 255; green, 255; blue, 255 }  ,draw opacity=1 ][line width=0.75]  (167,70.06) .. controls (167,50.73) and (182.67,35.06) .. (202,35.06) .. controls (221.33,35.06) and (237,50.73) .. (237,70.06) .. controls (237,89.39) and (221.33,105.06) .. (202,105.06) .. controls (182.67,105.06) and (167,89.39) .. (167,70.06) -- cycle ;
%Shape: Ellipse [id:dp19734961102385795] 
\draw  [color={rgb, 255:red, 255; green, 255; blue, 255 }  ,draw opacity=1 ][line width=0.75]  (202,56.06) .. controls (221.33,56.06) and (237,62.32) .. (237,70.06) .. controls (237,77.79) and (221.33,84.06) .. (202,84.06) .. controls (182.67,84.06) and (167,77.79) .. (167,70.06) .. controls (167,62.32) and (182.67,56.06) .. (202,56.06) -- cycle ;
%Straight Lines [id:da5462028174124043] 
\draw [color={rgb, 255:red, 255; green, 255; blue, 255 }  ,draw opacity=1 ][line width=1.5]    (209.43,61.88) -- (191.77,80.23) ;
\draw [shift={(189,83.11)}, rotate = 313.9] [fill={rgb, 255:red, 255; green, 255; blue, 255 }  ,fill opacity=1 ][line width=0.08]  [draw opacity=0] (11.61,-5.58) -- (0,0) -- (11.61,5.58) -- cycle    ;
\draw [shift={(212.2,59)}, rotate = 133.9] [fill={rgb, 255:red, 255; green, 255; blue, 255 }  ,fill opacity=1 ][line width=0.08]  [draw opacity=0] (11.61,-5.58) -- (0,0) -- (11.61,5.58) -- cycle    ;
%Curve Lines [id:da5059524634861983] 
\draw [color={rgb, 255:red, 255; green, 255; blue, 255 }  ,draw opacity=1 ][line width=1.5]    (183,31.11) .. controls (169.05,35.76) and (161.15,49.06) .. (161.73,66.18) ;
\draw [shift={(162,70.11)}, rotate = 263.99] [fill={rgb, 255:red, 255; green, 255; blue, 255 }  ,fill opacity=1 ][line width=0.08]  [draw opacity=0] (11.61,-5.58) -- (0,0) -- (11.61,5.58) -- cycle    ;

% Text Node
\draw (88,7) node [anchor=north west][inner sep=0.75pt]  [color={rgb, 255:red, 255; green, 255; blue, 255 }  ,opacity=1 ]  {$|0\rangle $};
% Text Node
\draw (90,110) node [anchor=north west][inner sep=0.75pt]  [color={rgb, 255:red, 255; green, 255; blue, 255 }  ,opacity=1 ]  {$|1\rangle $};
% Text Node
\draw (224,20) node [anchor=north west][inner sep=0.75pt]  [color={rgb, 255:red, 255; green, 255; blue, 255 }  ,opacity=1 ]  {$|1\rangle $};
% Text Node
\draw (161,94) node [anchor=north west][inner sep=0.75pt]  [color={rgb, 255:red, 255; green, 255; blue, 255 }  ,opacity=1 ]  {$|0\rangle $};


\end{tikzpicture}

\caption{Rotating the sphere}
\end{board}
\end{figure}


Practically, this is an observable that we specify by a rotation of the overall sphere. The points that end up at the poles of the rotated sphere become the measurement we obtain from looking at the qubit.

Mitiq provides an API to specify an observable. It takes a list of `PauliStrings`. These denote the rotations of the Bloch Sphere. In the Mitiq example, we have two qubits. The first `PauliString` applies Z-gates on both qubits (flipping around the Z-axis). The second `PauliString` applies a rotation around the X-axis on the first qubit by -1.75 (that is a little more than half the circle that would equal $\pi$ (around 3.14).

When we look at the observable, we can see that it outputs the compound rotation.

In [28]:
from mitiq import Observable, PauliString

obs = Observable(PauliString("ZZ"), PauliString("X", coeff=-1.75))
print(obs)

Z(0)*Z(1) + (-1.75+0j)*X(0)


So, with the observable at our disposal, let's return to the executor.

## Define An Executor - Revisited

The execute_with_noise_and_shots function requires the observable as a NumPy array. We get this representation by calling the `matrix` function of the observable object.

Next, we need to specify a noise_model. The noise_model tells the simulator what kind of noise to add to the simulation.

Qiskit provides the noise package to create a custom noise_model. We use it to add errors on single-qubit and two-qubit gates with a certain probability. This means that whenever we apply a gate of a specific kind, we will end up with a destroyed qubit state with the specified probability.

In [29]:
# 2. Define an executor
from mitiq.interface.mitiq_qiskit import qiskit_utils
from qiskit import QuantumCircuit, execute, Aer
import qiskit.providers.aer.noise as noise

# Error probabilities
prob_1 = 0.005  # 1-qubit gate
prob_2 = 0.01   # 2-qubit gate

# Depolarizing quantum errors
error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)

# Add errors to noise model
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

def sim_noise(qc):
    return qiskit_utils.execute_with_shots_and_noise(qc, obs.matrix(), noise_model, 4096)


Finally, we need to specify the number of shots we want to run the circuit. Anything beyond 1000 shots should work fine.

## (Near-Clifford) Simulator

The final component is a noise-free simulator. It is almost similar to the executor. The only difference is that it should not have any noise. We can simply use the `execute_with_shots` function.

In [30]:
def sim(qc):
    return qiskit_utils.execute_with_shots(qc, obs.matrix(), 4096)


## Run CDR

We're ready to run the CDR. We can use the rest of the example code as it is. We only need to plug in the functions we created.

We first compute the noiseless result.

In [32]:
ideal_measurement = obs.expectation(get_circuit(), sim).real
print("ideal_measurement = ",ideal_measurement)

ideal_measurement =  0.9644790763345432


Then, we compute the unmitigated noisy result.


In [33]:
unmitigated_measurement = obs.expectation(get_circuit(), sim_noise).real
print("unmitigated_measurement = ", unmitigated_measurement)

unmitigated_measurement =  0.8060289423652968


Next, we calculate the mitigated result from CDR.

In [34]:
from mitiq import cdr

mitigated_measurement = cdr.execute_with_cdr(
    get_circuit(),
    sim_noise,
    observable=obs.matrix(),
    simulator=sim,
    seed=0,
).real
print("mitigated_measurement = ", mitigated_measurement)

mitigated_measurement =  1.0128416969298948


Finally, we compare the results.

In [35]:
error_unmitigated = abs(unmitigated_measurement-ideal_measurement)
error_mitigated = abs(mitigated_measurement-ideal_measurement)

print("Error (unmitigated):", error_unmitigated)
print("Error (mitigated with CDR):", error_mitigated)

print("Relative error (unmitigated):", (error_unmitigated/ideal_measurement))
print("Relative error (mitigated with CDR):", error_mitigated/ideal_measurement)

print(f"Error reduction with CDR: {(error_unmitigated-error_mitigated)/error_unmitigated :.1%}.")

Error (unmitigated): 0.15845013396924645
Error (mitigated with CDR): 0.04836262059535157
Relative error (unmitigated): 0.16428571428571437
Relative error (mitigated with CDR): 0.05014377375520826
Error reduction with CDR: 69.5%.


A look at the results shows that CDR mitigates almost 90% of the errors that result from the noise.

Mitiq helped us to use CDR almost out of the box. We did not have to bother with the implementation of it at all. However, preparing the code to work with the API is a little bit tricky because the example seems to be outdated.